In [112]:
from sklearn.cluster import DBSCAN
import numpy as np


In [113]:
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.model_selection import KFold
# import numpy as np
import pandas as pd

In [114]:
from create_matrix import *

from tensor_custom_core import *
from create_matrix import *
from tensor_custom_core import *
from degree_days import dds
appliance_index = {appliance: APPLIANCES_ORDER.index(appliance) for appliance in APPLIANCES_ORDER}

APPLIANCES = ['fridge', 'hvac', 'wm', 'mw', 'oven', 'dw']
region = "SanDiego"
year = 2014

import os
from degree_days import dds
import autograd.numpy as np




def un_normalize(x, maximum, minimum):
    return (maximum - minimum) * x + minimum

In [150]:
def get_tensor(df):
    start, stop = 1, 13
    energy_cols = np.array(
        [['%s_%d' % (appliance, month) for month in range(start, stop)] for appliance in APPLIANCES_ORDER]).flatten()

    dfc = df.copy()

    df = dfc[energy_cols]

    tensor = df.values.reshape((len(df), 7, stop - start))
    return tensor


def create_region_df_dfc_static(region, year):
    df, dfc = create_matrix_single_region(region, year)
    tensor = get_tensor(df)
    static_region = df[['area', 'total_occupants', 'num_rooms']].copy()
    static_region['area'] = static_region['area'].div(4000)
    static_region['total_occupants'] = static_region['total_occupants'].div(8)
    static_region['num_rooms'] = static_region['num_rooms'].div(8)
    static_region =static_region.values
    return df, dfc, tensor, static_region


In [151]:
au_df, au_dfc, au_tensor, au_static = create_region_df_dfc_static('Austin', year)
sd_df, sd_dfc, sd_tensor, sd_static = create_region_df_dfc_static('SanDiego', year)

In [16]:
sd_agg = sd_df.loc[:, 'aggregate_1':'aggregate_12']
au_agg = au_df.loc[:, 'aggregate_1':'aggregate_12']

In [19]:
sd_agg = np.nan_to_num(sd_agg)
au_agg = np.nan_to_num(au_agg)

In [35]:
def distance(x, y):
    return np.linalg.norm(x - y)

In [192]:
def get_L(X):
    db = DBSCAN(eps=500, min_samples=1, metric=distance).fit(X)
    labels = db.labels_
    n_sample, n_feature = X.shape
    W = np.zeros((n_sample, n_sample))
    for i in range(n_sample):
        label = labels[i]
        neighbours = [j for j,x in enumerate(labels) if x == label]
        for k in neighbours:
            W[i][k] = 1
    K = np.dot(W, np.ones((n_sample, n_sample)))        
    D = np.diag(np.diag(K))
    return D-W

In [133]:
def cost_graph_laplacian(H, A, T, F, L, E_np_masked, K, lam1, lam2, case):
    HAT = multiply_case(H, A, T, case)
    mask = ~np.isnan(E_np_masked)
    error_1 = (HAT - E_np_masked)[mask].flatten()
    
    HF = np.einsum('na, ab->nb', H, F)
    mask_static = ~np.isnan(k)
    error_2 = (HF - K)[mask_static].flatten()
    
    HTL = np.dot(H.T, L)
    HTLH = np.dot(HTL, H)
    error_3 = np.trace(HTLH)
    
    return np.sqrt((error_1**2).mean()) + lam1 * np.sqrt((error_2**2).mean()) + lam2 * error_3

In [198]:
def cost_graph_laplacian_2(H, A, T, H_f, F, L, E_np_masked, K, lam1, lam2, case):
    HAT = multiply_case(H, A, T, case)
    mask = ~np.isnan(E_np_masked)
    error_1 = (HAT - E_np_masked)[mask].flatten()
    
    HF = np.einsum('na, ab->nb', H_f, F)
    mask_static = ~np.isnan(k)
    error_2 = (HF - K)[mask_static].flatten()
    
    HTL = np.dot(H_f.T, L)
    HTLH = np.dot(HTL, H_f)
    error_3 = np.trace(HTLH)
    
    return np.sqrt((error_1**2).mean()) + lam1 * np.sqrt((error_2**2).mean()) + lam2 * error_3

In [136]:
mg = multigrad(cost_graph_laplacian, argnums=[0, 1, 2, 3])

H = np.random.rand(n_sample_sd, 3)
A = np.random.rand(7, 3, 3)
T = np.random.rand(12, 3)
F = np.random.rand(3, 12)
del_h, del_a, del_t, del_f = mg(H, A, T, F, L_sd, sd_tensor, sd_agg, 1, 1, 2)

In [138]:
def learn_HAT_adagrad_graph(case, E_np_masked, L, K, a, b, num_iter=2000, lr=0.1, dis=False, lam1 = 1, lam2 = 1, H_known=None,
                      A_known=None, T_known=None, random_seed=0, eps=1e-8, penalty_coeff=0.0):

    cost = cost_graph_laplacian
    mg = multigrad(cost, argnums=[0, 1, 2, 3])

    params = {}
    params['M'], params['N'], params['O'] = E_np_masked.shape
    params['a'] = a
    params['b'] = b
    H_dim_chars = list(cases[case]['HA'].split(",")[0].strip())
    H_dim = tuple(params[x] for x in H_dim_chars)
    A_dim_chars = list(cases[case]['HA'].split(",")[1].split("-")[0].strip())
    A_dim = tuple(params[x] for x in A_dim_chars)
    T_dim_chars = list(cases[case]['HAT'].split(",")[1].split("-")[0].strip())
    T_dim = tuple(params[x] for x in T_dim_chars)
    
    H = np.random.rand(*H_dim)
    A = np.random.rand(*A_dim)
    T = np.random.rand(*T_dim)
    F = np.random.rand(a, 12)

    sum_square_gradients_H = np.zeros_like(H)
    sum_square_gradients_A = np.zeros_like(A)
    sum_square_gradients_T = np.zeros_like(T)
    sum_square_gradients_F = np.zeros_like(F)

    Hs = [H.copy()]
    As = [A.copy()]
    Ts = [T.copy()]
    Fs = [F.copy()]
    
    costs = [cost(H, A, T, F, L, E_np_masked, K, lam1, lam2, case)]
    
#     if 'penalty' not in cost_function:
#         costs = [cost(H, A, T, H_1, F, E_np_masked, static, case)]
#     else:
#         costs = [cost(H, A, T, E_np_masked, 2, penalty_coeff)]
    HATs = [multiply_case(H, A, T, case)]

    # GD procedure
    for i in range(num_iter):
        del_h, del_a, del_t, del_f = mg(H, A, T, F, L, E_np_masked, K, lam1, lam2, case)
        sum_square_gradients_H += eps + np.square(del_h)
        sum_square_gradients_A += eps + np.square(del_a)
        sum_square_gradients_T += eps + np.square(del_t)
        sum_square_gradients_F += eps + np.square(del_f)

        lr_h = np.divide(lr, np.sqrt(sum_square_gradients_H))
        lr_a = np.divide(lr, np.sqrt(sum_square_gradients_A))
        lr_t = np.divide(lr, np.sqrt(sum_square_gradients_T))
        lr_f = np.divide(lr, np.sqrt(sum_square_gradients_F))

        H -= lr_h * del_h
        A -= lr_a * del_a
        T -= lr_t * del_t
        F -= lr_f * del_f
        # Projection to known values
        if H_known is not None:
            H = set_known(H, H_known)
        if A_known is not None:
            A = set_known(A, A_known)
        if T_known is not None:
            T = set_known(T, T_known)
        # Projection to non-negative space
        H[H < 0] = 1e-8
        A[A < 0] = 1e-8
        T[T < 0] = 1e-8
        F[F < 0] = 1e-8

        As.append(A.copy())
        Ts.append(T.copy())
        Hs.append(H.copy())
        Fs.append(F.copy())
        
        costs.append(cost(H, A, T, F, L, E_np_masked, K, lam1, lam2, case))
        
#         if 'penalty' not in cost_function:
#             costs.append(cost(H, A, T, H_1, F, E_np_masked, static, case))
#         else:
#             costs.append(cost(H, A, T, F, E_np_masked, static, case))
        HATs.append(multiply_case(H, A, T, case))
        if i % 500 == 0:
            if dis:
                print(cost(H, A, T, F, L, E_np_masked, K, lam1, lam2, case))
    return H, A, T, F, Hs, As, Ts, Fs, HATs, costs

In [202]:
def learn_HAT_adagrad_graph_2(case, E_np_masked, L, K, a, b, c, num_iter=2000, lr=0.1, dis=False, lam1 = 1, lam2 = 1, H_known=None,
                      A_known=None, T_known=None, random_seed=0, eps=1e-8, penalty_coeff=0.0):

    cost = cost_graph_laplacian_2
    mg = multigrad(cost, argnums=[0, 1, 2, 3, 4])

    params = {}
    params['M'], params['N'], params['O'] = E_np_masked.shape
    params['a'] = a
    params['b'] = b
    H_dim_chars = list(cases[case]['HA'].split(",")[0].strip())
    H_dim = tuple(params[x] for x in H_dim_chars)
    A_dim_chars = list(cases[case]['HA'].split(",")[1].split("-")[0].strip())
    A_dim = tuple(params[x] for x in A_dim_chars)
    T_dim_chars = list(cases[case]['HAT'].split(",")[1].split("-")[0].strip())
    T_dim = tuple(params[x] for x in T_dim_chars)
    
    H = np.random.rand(*H_dim)
    A = np.random.rand(*A_dim)
    T = np.random.rand(*T_dim)
    H_f = np.random.rand(params['M'], c)
    F = np.random.rand(c, 12)

    sum_square_gradients_H = np.zeros_like(H)
    sum_square_gradients_A = np.zeros_like(A)
    sum_square_gradients_T = np.zeros_like(T)
    sum_square_gradients_H_f = np.zeros_like(H_f)
    sum_square_gradients_F = np.zeros_like(F)

    Hs = [H.copy()]
    As = [A.copy()]
    Ts = [T.copy()]
    H_fs = [H_f.copy()]
    Fs = [F.copy()]
    
    costs = [cost(H, A, T, H_f, F, L, E_np_masked, K, lam1, lam2, case)]
    
#     if 'penalty' not in cost_function:
#         costs = [cost(H, A, T, H_1, F, E_np_masked, static, case)]
#     else:
#         costs = [cost(H, A, T, E_np_masked, 2, penalty_coeff)]
    HATs = [multiply_case(H, A, T, case)]

    # GD procedure
    for i in range(num_iter):
        del_h, del_a, del_t, del_h_f, del_f = mg(H, A, T, H_f, F, L, E_np_masked, K, lam1, lam2, case)
        sum_square_gradients_H += eps + np.square(del_h)
        sum_square_gradients_A += eps + np.square(del_a)
        sum_square_gradients_T += eps + np.square(del_t)
        sum_square_gradients_H_f += eps + np.square(del_h_f)
        sum_square_gradients_F += eps + np.square(del_f)

        lr_h = np.divide(lr, np.sqrt(sum_square_gradients_H))
        lr_a = np.divide(lr, np.sqrt(sum_square_gradients_A))
        lr_t = np.divide(lr, np.sqrt(sum_square_gradients_T))
        lr_h_f = np.divide(lr, np.sqrt(sum_square_gradients_H_f))
        lr_f = np.divide(lr, np.sqrt(sum_square_gradients_F))

        H -= lr_h * del_h
        A -= lr_a * del_a
        T -= lr_t * del_t
        H_f -= lr_h_f * del_h_f
        F -= lr_f * del_f
        # Projection to known values
        if H_known is not None:
            H = set_known(H, H_known)
        if A_known is not None:
            A = set_known(A, A_known)
        if T_known is not None:
            T = set_known(T, T_known)
        H = set_known(H, H_f)
        # Projection to non-negative space
        H[H < 0] = 1e-8
        A[A < 0] = 1e-8
        T[T < 0] = 1e-8
        H_f[H_f < 0] = 1e-8
        F[F < 0] = 1e-8

        As.append(A.copy())
        Ts.append(T.copy())
        Hs.append(H.copy())
        H_fs.append(H_f.copy())
        Fs.append(F.copy())
        
        costs.append(cost(H, A, T, H_f, F, L, E_np_masked, K, lam1, lam2, case))
        
#         if 'penalty' not in cost_function:
#             costs.append(cost(H, A, T, H_1, F, E_np_masked, static, case))
#         else:
#             costs.append(cost(H, A, T, F, E_np_masked, static, case))
        HATs.append(multiply_case(H, A, T, case))
        if i % 500 == 0:
            if dis:
                print(cost(H, A, T, H_f, F, L, E_np_masked, K, lam1, lam2, case))
    return H, A, T, F, Hs, As, Ts, Fs, HATs, costs

In [203]:
a = 5
b = 3
c = 3
H_au, A_au, T_au, F_au, Hs, As, Ts, Fs, HATs, costs = learn_HAT_adagrad_graph_2(case, au_tensor, L_au, au_agg, a, b, c, num_iter=2000, lr=0.1, dis=True, random_seed=0, eps=1e-8, penalty_coeff=0.0)



18248.1643894
959.262843555
907.539797028
881.272431918


In [193]:
n_splits = 10
case = 2
a = 5
b = 3
c = 3
iters = 2000

# H_au, A_au, T_au, F_au = learn_HAT_graph(2, au_tensor, static_au, sim_au, a, b, num_iter=iters, dis=True, T_known = np.ones(12).reshape(-1, 1))
L_au = get_L(au_agg)
L_sd = get_L(sd_agg)
H_au, A_au, T_au, F_au, Hs, As, Ts, Fs, HATs, costs = learn_HAT_adagrad_graph(case, au_tensor, L_au, au_agg, a, b, c, num_iter=2000, lr=0.1, dis=True, random_seed=0, eps=1e-8, penalty_coeff=0.0)
pred_normal = {}
pred_transfer = {}
for random_seed in range(10):
    pred_normal[random_seed] = {}
    pred_transfer[random_seed] = {}
    
    for appliance in APPLIANCES_ORDER:
        pred_normal[random_seed][appliance] = {f:[] for f in range(10, 110, 10)}
        pred_transfer[random_seed][appliance] = {f:[] for f in range(10, 110, 10)}

kf = KFold(n_splits=n_splits)
for random_seed in range(1):
    print "random seed: ", random_seed
    np.random.seed(random_seed)
    for train_percentage in range(10, 20, 10):
        print "training percentage: ", train_percentage
        rd = 0
        
        for train_max, test in kf.split(sd_df):
            print "round: ", rd
           
            
            num_train = int((train_percentage*len(train_max)/100)+0.5)
            num_test = len(test)
            
            # get the random training data from train_max based on then random seed
            if train_percentage==100:
                train = train_max
            else:
                train, _ = train_test_split(train_max, train_size = train_percentage/100.0)
            
            # get the index of training and testing data
            train_ix = sd_df.index[train]
            test_ix = sd_df.index[test]
            print "test_ix: ", test_ix
            
            # create the tensor
            print test, train
            train_test_ix = np.concatenate([test_ix, train_ix])
            df_t, dfc_t = sd_df.ix[train_test_ix], sd_dfc.ix[train_test_ix]
            tensor = get_tensor(df_t)
            
            ############################################################################################
            # Normal learning: no constant constraint, no A_known, with learn_HAT
            ############################################################################################
            tensor_copy = tensor.copy()
            tensor_copy[:num_test, 1:, :] = np.NaN
            agg = sd_agg[np.concatenate([test, train])]
            L = L_sd[np.ix_(np.concatenate([test, train]), np.concatenate([test, train]))]
            
#             H, A, T, F = learn_HAT_graph(2, tensor_copy, static_sd[np.concatenate([test, train])], sim_sd, a, b, num_iter=iters,dis=True, T_known = np.ones(12).reshape(-1, 1))
            H, A, T, F, Hs, As, Ts, Fs, HATs, costs = learn_HAT_adagrad_graph(case, tensor_copy, L, agg, a, b, num_iter=20000, lr=0.1, dis=True)
            
            # get the prediction
            HAT = multiply_case(H, A, T, case)
            for appliance in APPLIANCES_ORDER:
                pred_normal[random_seed][appliance][train_percentage].append(pd.DataFrame(HAT[:num_test, appliance_index[appliance], :], index=test_ix))
                       
            
            ############################################################################################
            # transfer learning: constant constraint, with A_known = A_a_const, with learn_HAT_constant
            ############################################################################################
            tensor_copy = tensor.copy()
            tensor_copy[:num_test, 1:, :] = np.NaN
            agg = sd_agg[np.concatenate([test, train])]
            L = L_sd[np.ix_(np.concatenate([test, train]), np.concatenate([test, train]))]
            
#             H, A, T, F = learn_HAT_graph(2, tensor_copy, static_sd[np.concatenate([test, train])], sim_sd, a, b, num_iter=iters,dis=True, T_known = np.ones(12).reshape(-1, 1))
            H, A, T, F, Hs, As, Ts, Fs, HATs, costs = learn_HAT_adagrad_graph(case, tensor_copy, L, agg, a, b, num_iter=20000, lr=0.1, dis=True,A_known = A_au)
            
            # get the prediction
            HAT = multiply_case(H, A, T, case)
            for appliance in APPLIANCES_ORDER:
                pred_transfer[random_seed][appliance][train_percentage].append(pd.DataFrame(HAT[:num_test, appliance_index[appliance], :], index=test_ix))
            
            
            rd += 1

30235.9467028
1107.82281384
1079.91045284
1059.14678675
random seed:  0
training percentage:  10
round:  0
test_ix:  Int64Index([54, 203, 527, 1450], dtype='int64')
[0 1 2 3] [37  7  4]
934.573835619
696.004741317
682.555704549
670.515186579
646.688595905
598.205601354
565.08163453
547.151821617
533.956407191
522.770805604
512.60041077
502.734199582
492.914650444
482.475731177
474.060697481
466.428852012
459.479163889
452.505946943
445.410486677
438.568530823
432.361923117
426.431463071
420.780723164
415.43228412
410.311694766
405.400262146
400.738491806
396.368427819
392.272007669
388.419236529
384.816713925
381.45750651
378.314248983
375.43591081
372.798743024
370.435475166
368.338862361
366.414113572
364.643720714
363.016589821
950.158841422
688.538329959
666.704785665
647.170267957
632.406603195
621.717896886
613.768323942
607.591793523
602.589587853
598.460605326
594.992191033
591.970437563
589.279532132
586.81387729
584.505475227
582.300020572
580.323834079
578.569721085
576.9282

495.322621789
493.67749886
492.155553019
490.742591736
489.423773976
488.187579251
487.024731551
485.928238956
484.89372089
483.911138591
482.976440938
482.08424009
481.231357351
480.412304189
479.626543755
478.871422292
478.142271762
477.438981867
476.759863813
476.10185048
475.464222907
474.846044882
474.251117029
473.6770149
473.118715007
round:  7
test_ix:  Int64Index([7639, 8061, 8342, 8574], dtype='int64')
[28 29 30 31] [13 23 10]
775.95935447
569.563366448
523.906733211
488.532111705
474.585402521
467.259730962
462.668386369
459.248885378
456.31356355
453.384499526
449.812752241
444.949601055
438.535895232
431.08519836
423.530817686
416.574948739
410.717155704
405.95506017
401.903256983
398.49873902
395.577506209
393.024297526
390.754908688
388.714693495
386.865532784
385.173025986
383.617028601
382.182463631
380.85804644
379.635416726
378.496560677
377.433974755
376.475379596
375.58956219
374.766029364
373.999663981
373.284767369
372.614567905
371.984939634
371.389609093
773.73

In [195]:
out = {}

out_normal = {}
out_transfer = {}

for random_seed in range(1):
    out_normal[random_seed] = {}   
    for appliance in APPLIANCES_ORDER[1:]:
        out_normal[random_seed][appliance] = {}
        for f in range(10,20,10):
            s = pd.concat(pred_normal[random_seed][appliance][f]).ix[sd_df.index]
            if appliance=="hvac":
                out_normal[random_seed][appliance][f] = compute_rmse_fraction(appliance,s[range(4, 10)],'SanDiego')[2]
            else:   
                out_normal[random_seed][appliance][f] = compute_rmse_fraction(appliance, s,'SanDiego')[2]
for random_seed in range(1):
    out_transfer[random_seed] = {}   
    for appliance in APPLIANCES_ORDER[1:]:
        out_transfer[random_seed][appliance] = {}
        for f in range(10,20,10):
            s = pd.concat(pred_transfer[random_seed][appliance][f]).ix[sd_df.index]
            if appliance=="hvac":
                out_transfer[random_seed][appliance][f] = compute_rmse_fraction(appliance,s[range(4, 10)],'SanDiego')[2]
            else:   
                out_transfer[random_seed][appliance][f] = compute_rmse_fraction(appliance, s,'SanDiego')[2]

/Users/xox/Library/Python/2.7/lib/python/site-packages/ipykernel_launcher.py:11: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  # This is added back by InteractiveShellApp.init_path()
/Users/xox/Library/Python/2.7/lib/python/site-packages/ipykernel_launcher.py:21: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated


In [196]:
out_normal

{0: {'dw': {10: 2.4456218169391195},
  'fridge': {10: 10.205282875095655},
  'hvac': {10: 20.225307489370415},
  'mw': {10: 2.6619896148991034},
  'oven': {10: 4.7118170758405231},
  'wm': {10: 2.5579010092258456}}}

In [197]:
out_transfer

{0: {'dw': {10: 2.0328016677054781},
  'fridge': {10: 10.963923958341976},
  'hvac': {10: 18.940246348263724},
  'mw': {10: 2.3112330677464188},
  'oven': {10: 2.48639255792042},
  'wm': {10: 0.9714782242877904}}}

In [129]:
HAT = multiply_case(H, A, T, case)

In [130]:
pred = {}

for appliance_name, appliance_id in appliance_index.iteritems():
    pred[appliance_name] = []
    pred[appliance_name].append(pd.DataFrame(HAT[:,appliance_id,:], index=sd_df.index))

In [131]:
from sklearn.model_selection import train_test_split, KFold
from common import compute_rmse
from common import compute_rmse_fraction
out = {}
for appliance in APPLIANCES_ORDER[1:]:
    out[appliance] = {}
    s = pd.concat(pred[appliance]).ix[sd_df.index]
    if appliance=="hvac":
        out[appliance] = compute_rmse_fraction(appliance,s[range(4, 10)],'SanDiego')[2]
    else:   
        out[appliance] = compute_rmse_fraction(appliance, s,'SanDiego')[2]

/Users/xox/Library/Python/2.7/lib/python/site-packages/ipykernel_launcher.py:7: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  import sys


In [132]:
out

{'dw': 1.8424540871334227,
 'fridge': 6.1113753542608542,
 'hvac': 21.221250060053698,
 'mw': 1.7863016435529679,
 'oven': 2.1007380892141541,
 'wm': 0.55487012541894776}